# Paleo-Detector Analysis Pipeline

This notebook provides a complete, end-to-end workflow for the phenomenological analysis of paleo-detectors. It is designed to calculate the expected nuclear recoil track spectra from both external cosmic ray muon signals and various internal/atmospheric backgrounds.

The pipeline is built around the *mineral_utils.py* module, which contains the *Paleodetector* class. This class acts as the physics engine, handling all low-level data processing and calculations. The notebook serves as a high-level interface to configure and orchestrate the analysis.

### Workflow Overview:

**1. Configuration:** All analysis parameters are defined in the configuration cells. This includes selecting the mineral from a library, defining the geological history of the sample (age, exposure time, deposition rate), and setting up the astrophysical scenarios for the cosmic ray flux.

**2. Initialization:** A Paleodetector object is created. This object automatically loads and caches all necessary input data, such as SRIM tables for ion ranges and pre-computed Geant4 results for muon interactions.

**3. Background Calculation:** The notebook calculates the track spectra for the primary background sources:

- Spontaneous Fission: Tracks from the spontaneous fission of Uranium-238 impurities within the mineral.

- Radiogenic Neutrons: Tracks from neutrons produced by (α,n) reactions in the surrounding rock.

- Neutrinos: Tracks from coherent elastic neutrino-nucleus scattering.

**7. Signal Calculation:** For each defined astrophysical scenario, the notebook calculates the track spectrum from cosmic ray muons. This core step includes a sophisticated model for a time-variant cosmic ray flux and the continuous deposition of overburden, which attenuates the signal over the exposure window. The time integration is performed in parallel for efficiency.

**8. Analysis & Plotting:** The final signal and background components are combined to produce the summary plots, showing the total expected number of tracks as a function of track length.


In [ ]:
# --- Core Libraries ---
import numpy as np
from matplotlib import pyplot as plt

# --- Custom Utility Module ---
from mineral_utils import Paleodetector

print("Libraries imported successfully.")

In [ ]:
CONFIG = {
    # --- Mineral and File Configuration ---
    "mineral_name": "Olivine",  # Must match a key in MINERAL_LIBRARY below
    
    # --- Geological & Sample Parameters ---
    "sample_mass_kg": 0.00001,       # Sample mass in kg (e.g., 0.01 for 10g)
    "total_age_myr" : 0.04,
    #"total_exposure_kyr": 5900.,         # Total age of the sample in millions of years
    #"exposure_window_kyr": 600.,  # Duration of CR signal exposure in kyr
    "total_exposure_kyr": 38.1,         # Total age of the sample in kyr
    "exposure_window_kyr": 38.,  # Duration of CR signal exposure in kyr


    # --- Continuous Deposition Model Parameters ---
    "deposition_rate_m_kyr": 0., 
    "continuous_overburden_density_g_cm3": 2.16,
    #"event_overburden_density_g_cm3": 1.,
    "event_overburden_density_g_cm3": 2.7,
    "event_overburden_depth": 10.,
    "initial_depth_m": 0.,

    # --- Plotting & Binning Configuration ---
    "n_bins": 60,
    "x_min_log_nm": 2.5, # Min track length in log10(nm)
    "x_max_log_nm": 5., # Max track length in log10(nm)

    # Parameters derived from Geant4 simulation setup
    "geant4_energy_bins_gev": [0.01      , 0.01165914, 0.01359356, 0.01584893, 0.0184785 ,
        0.02154435, 0.02511886, 0.02928645, 0.03414549, 0.03981072,
        0.04641589, 0.05411695, 0.06309573, 0.07356423, 0.08576959,
        0.1       , 0.11659144, 0.13593564, 0.15848932, 0.18478498,
        0.21544347, 0.25118864, 0.29286446, 0.34145489, 0.39810717,
        0.46415888, 0.54116953, 0.63095734, 0.73564225, 0.8576959 ,
        1.0000, 1.1909, 1.4183, 1.6891, 2.0116, 2.3956, 2.8530, 3.3977,
        4.0464, 4.8189, 5.7390, 6.8347, 8.1396, 9.6936, 11.5443, 13.7484, 
        16.3732, 19.4993, 23.2221, 27.6557, 32.9358, 39.2240, 46.7127, 
        55.6312, 66.2524, 78.9015, 93.9656, 111.9057, 133.2710, 158.7153, 
        189.0176, 225.1053, 268.0829, 319.2659, 380.2209, 452.8135, 539.2657, 
        642.2235, 764.8383, 910.8629, 1084.7669, 1291.8731, 1538.5204, 1832.2582, 
        2182.0771, 2598.6842, 3094.8308, 3685.7029, 4389.3856, 5227.4170, 
        6225.4472, 7414.0236, 8829.5257, 10515.2787, 12522.8794, 14913.7758, 
        17761.1475, 21152.1459, 25190.5612, 30000.0000, 100000],

    "mineral_target_thickness_cm": 1000, # Thickness of the target in the Geant4 simulation in cm
    "total_simulated_muons": 3e4,        # Number of muons simulated in each Geant4 run, for weighting
}

In [ ]:
scenario_config_simple = {
    'name': 'simple',
    'event_fluxes': {
        0.: ('H3a', 'H3a'),
    }
}

In [ ]:
# --- Mineral Library ---
# To analyze a new mineral, add its properties here and ensure the
# corresponding SRIM and Geant4 data files exist in the Data/ directory.
MINERAL_LIBRARY = {
    "Halite": {
        "name": "Halite",
        "shortname": "Hal",
        "composition": "Na-Cl",
        "nuclei": ["Na", "Cl"],
        "stoich": [1, 1],
        "uranium_concentration_g_g": 1e-9,
        "density_g_cm3": 2.16,
    },
    "Olivine": {
        "name": "Olivine",
        "shortname": "Oli",
        "composition": "Mg-Fe-Si- O",
        "nuclei": ["Mg", "Fe", "Si", "O"],
        "stoich": [1.8, 0.2, 1, 4],
        "uranium_concentration_g_g": 1e-7,
        "density_g_cm3": 3.8,
    },
    "Quartz": {
        "name": "Quartz",
        "shortname": "Qz",
        "composition": "Si- O",
        "nuclei": ["Si", "O"],
        "stoich": [1, 2],
        "uranium_concentration_g_g": 1e-5,
        "density_g_cm3": 2.6,
    },
    "Spinel": {
        "name": "Spinel",
        "shortname": "Spi",
        "composition": "Mg-Al- O",
        "nuclei": ["Mg", "Al", "O"],
        "stoich": [1, 2, 4],
        "uranium_concentration_g_g": 1e-7,
        "density_g_cm3": 3.7,
    }
}

In [ ]:
x_bins = np.logspace(CONFIG["x_min_log_nm"], CONFIG["x_max_log_nm"], CONFIG["n_bins"])
x_mids = x_bins[:-1] + np.diff(x_bins) / 2

In [ ]:
# --- Setup & Verification ---
mineral_config = MINERAL_LIBRARY.get(CONFIG["mineral_name"])
if not mineral_config:
    raise ValueError(f"Mineral '{CONFIG['mineral_name']}' not found in MINERAL_LIBRARY.")

mineral = Paleodetector(mineral_config)

## Calculation of Track Spectra

With the Paleodetector object initialized and all necessary data loaded, we can now proceed with the core physics calculations. The following cells will compute the differential track rate (dR/dx) for each relevant physical process and then integrate these rates over the sample's lifetime to find the total expected number of tracks in each track length bin.

### Calculation Steps:

**1. Backgrounds:** We first calculate the spectra for the steady-state backgrounds that accumulate over the entire geological age of the sample. These include spontaneous fission, radiogenic neutrons, and atmospheric neutrinos.

**2. Muon Signal:** Next, we compute the primary signal from cosmic ray muons. The integrate_muon_signal_spectrum_parallel function is used for this. It performs a numerical integration over the specified exposure_window_kyr, accounting for two key time-dependent effects at each step:

- The evolution of the cosmic ray flux, which is interpolated from the files defined in the scenario_config.

- The increase in shielding due to the continuous deposition_rate_m_kyr.

**3. Final Plot:** Finally, all calculated track components (signals and backgrounds) are summed and plotted to visualize the expected final track spectrum in the detector.

In [ ]:
total_nu_tracks = mineral.integrate_nu_spectrum(x_bins, CONFIG['total_age_myr'], CONFIG['sample_mass_kg'], flux_name="all")

In [ ]:
total_fission_tracks = mineral.integrate_fission_spectrum(x_bins, CONFIG['total_age_myr'], CONFIG['sample_mass_kg'])

In [ ]:
total_neutron_tracks = mineral.integrate_neutron_spectrum(x_bins, CONFIG['total_age_myr'], CONFIG['sample_mass_kg'])

In [ ]:
montcineyre = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=7.1,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
lavache = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=8.6,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
pariou = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=9.9,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
dome = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=11.,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
come = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=13.7,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
lemptegy = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=30.,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
laschamp = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    exposure_window_kyr=40.,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    nsteps=10,
)

In [ ]:
post_scenario = mineral.integrate_muon_signal_spectrum_parallel(
    x_bins=x_bins,
    scenario_config=scenario_config_simple,
    energy_bins_gev=CONFIG["geant4_energy_bins_gev"],
    #exposure_window_kyr=CONFIG["total_exposure_kyr"] - CONFIG["exposure_window_kyr"],
    exposure_window_kyr=0.005,
    sample_mass_kg=CONFIG["sample_mass_kg"],
    #initial_depth=CONFIG["event_overburden_depth"] * CONFIG["event_overburden_density_g_cm3"] + CONFIG["deposition_rate_m_kyr"] * CONFIG["exposure_window_kyr"] * CONFIG["continuous_overburden_density_g_cm3"],
    nsteps=10,
)

In [ ]:
total_muon_tracks = {
    "Montcineyre": montcineyre,
    "Lavache": lavache,
    "Pariou": pariou,
    "Dome": dome,
    "Come": come,
    "Lemptegy": lemptegy,
    "Laschamp": laschamp,
    "Modern": post_scenario
}

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
mineral.plot_tracks(x_bins, muon_tracks=total_muon_tracks, fission_tracks=total_fission_tracks, nu_tracks=total_nu_tracks, neutron_tracks=total_neutron_tracks, ax=ax)
ax.set_xscale('linear')
#ax.set_yscale('linear')
plt.show()